In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.sequence import pad_sequences

import os

In [2]:
imdb_train, ds_info = tfds.load(name="imdb_reviews",
                                split="train",
                                with_info=True, as_supervised=True)

imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE1ZH6B/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE1ZH6B/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE1ZH6B/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### Keras Tokenizer

In [ ]:
X_train = list(map(lambda x: x[0].numpy().decode('utf-8'), imdb_train))
y_train = list(map(lambda x: x[1], imdb_train))

In [ ]:
%%time
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                  lower=True,
                                                  split=" ",)
tokenizer.fit_on_texts(X_train)

X_encoded = tokenizer.texts_to_sequences(X_train)
X_encoded = pad_sequences(X_encoded, padding="post", maxlen=150) 

Wall time: 9.88 s


### TFDS Tokenizer

In [ ]:
tokenizer_tfds = tfds.deprecated.text.Tokenizer()
vocabulary_set = set()
MAX_TOKENS = 0
for example, label in imdb_train:
    some_tokens = tokenizer_tfds.tokenize(example.numpy())
    if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
    vocabulary_set.update(some_tokens)
    
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, lowercase=True, tokenizer=tokenizer_tfds)

vocab_size = imdb_encoder.vocab_size
print(vocab_size, MAX_TOKENS)

93931 2525


In [ ]:
from tensorflow.keras.preprocessing import sequence

def encode_pad_transform(sample):
    
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post',maxlen=150)
    return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
    
    encoded = tf.py_function(encode_pad_transform,
                             inp=[sample],
                             Tout=(tf.int64))
    
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label 

In [ ]:
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

### Loading pre-trained GloVe embeddings

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
from google.colab import drive

drive.mount('drive')
glove_path = 'drive/MyDrive/GloVe/glove.6B.50d.txt'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
dict_w2v = {}
with open(glove_path, "rb") as file:
    for line in file:
        tokens = line.split()
        word = tokens[0].decode('utf-8')
        vector = np.array(tokens[1:], dtype=np.float32)
        
        if vector.shape[0] == 50:
            dict_w2v[word] = vector
        else:
            print("There was an issue with " + word)
            
# let's check the vocabulary size
print("Dictionary Size: ", len(dict_w2v))

Dictionary Size:  400000


In [ ]:
vocab_size = imdb_encoder.vocab_size
embedding_dim = 50
embedding_matrix = np.zeros((vocab_size, embedding_dim))

### Tfds

In [ ]:
unk_cnt = 0
unk_set = set()
for word in imdb_encoder.tokens:
    embedding_vector = dict_w2v.get(word)
    
    if embedding_vector is not None:
        tkn_id = imdb_encoder.encode(word)[0]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)
# Print how many weren't found
print("Total unknown words: ", unk_cnt)

Total unknown words:  14553


### Keras

In [ ]:
unk_cnt = 0
unk_set = set()
for word in list(tokenizer.word_index.keys()):
    embedding_vector = dict_w2v.get(word)
    
    if embedding_vector is not None:
        tkn_id = tokenizer.word_index[word]
        embedding_matrix[tkn_id] = embedding_vector
    else:
        unk_cnt += 1
        unk_set.add(word)
# Print how many weren't found
print("Total unknown words: ", unk_cnt)

Total unknown words:  28423


### Build model

In [ ]:
vocab_size = imdb_encoder.vocab_size # len(chars)
# Number of RNN units
rnn_units = 64
#batch size
BATCH_SIZE=100

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size, train_emb=False):
    model = tf.keras.Sequential([
        Embedding(vocab_size, embedding_dim, mask_zero=True,
                  weights=[embedding_matrix], trainable=train_emb),
        Bidirectional(LSTM(rnn_units, return_sequences=True,
                           dropout=0.5)),
        Bidirectional(LSTM(rnn_units, dropout=0.25)),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
model_fe = build_model_bilstm(
    vocab_size = vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model_fe.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_4 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 4,854,375
Trainable params: 157,825
Non-trainable params: 4,696,550
_________________________________________________________________


In [ ]:
model_fe.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)
model_fe.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 33s 82ms/step - loss: 0.6347 - accuracy: 0.6282 - precision: 0.6288 - recall: 0.6348
Epoch 2/10
250/250 [==============================] - 22s 90ms/step - loss: 0.5586 - accuracy: 0.7157 - precision: 0.7223 - recall: 0.7025
Epoch 3/10
250/250 [==============================] - 23s 90ms/step - loss: 0.5194 - accuracy: 0.7432 - precision: 0.7470 - recall: 0.7379
Epoch 4/10
250/250 [==============================] - 22s 90ms/step - loss: 0.4896 - accuracy: 0.7647 - precision: 0.7644 - recall: 0.7665
Epoch 5/10
250/250 [==============================] - 23s 90ms/step - loss: 0.5145 - accuracy: 0.7462 - precision: 0.7491 - recall: 0.7424
Epoch 6/10
250/250 [==============================] - 22s 90ms/step - loss: 0.4608 - accuracy: 0.7800 - precision: 0.7829 - recall: 0.7757
Epoch 7/10
250/250 [==============================] - 23s 91ms/step - loss: 0.4534 - accuracy: 0.7883 - precision: 0.7968 - recall: 0.7752
Epoch 8/10
250/250 [=======

### Fine-tuning

In [ ]:
model_ft = build_model_bilstm(
 vocab_size=vocab_size,
 embedding_dim=embedding_dim,
 rnn_units=rnn_units,
 batch_size=BATCH_SIZE,
 train_emb=True)
model_ft.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 50)          4696550   
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 128)         58880     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 4,854,375
Trainable params: 4,854,375
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_ft.compile(loss='binary_crossentropy',
 optimizer='adam',
 metrics=['accuracy', 'Precision', 'Recall'])
model_ft.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 44s 125ms/step - loss: 0.6129 - accuracy: 0.6483 - precision: 0.6497 - recall: 0.6520
Epoch 2/10
250/250 [==============================] - 33s 131ms/step - loss: 0.4159 - accuracy: 0.8108 - precision: 0.8106 - recall: 0.8120
Epoch 3/10
250/250 [==============================] - 33s 131ms/step - loss: 0.3367 - accuracy: 0.8522 - precision: 0.8549 - recall: 0.8491
Epoch 4/10
250/250 [==============================] - 33s 130ms/step - loss: 0.2909 - accuracy: 0.8764 - precision: 0.8749 - recall: 0.8789
Epoch 5/10
250/250 [==============================] - 33s 131ms/step - loss: 0.2436 - accuracy: 0.9008 - precision: 0.9008 - recall: 0.9011
Epoch 6/10
250/250 [==============================] - 33s 131ms/step - loss: 0.2209 - accuracy: 0.9113 - precision: 0.9097 - recall: 0.9135
Epoch 7/10
250/250 [==============================] - 33s 131ms/step - loss: 0.1858 - accuracy: 0.9259 - precision: 0.9270 - recall: 0.9250
Epoch 8/10
250/250 [

### BERT

In [3]:
!pip install transformers==3.0.2

     |████████████████████████████████| 778kB 8.9MB/s 
     |████████████████████████████████| 3.0MB 52.3MB/s 
     |████████████████████████████████| 901kB 49.7MB/s 
     |████████████████████████████████| 1.2MB 49.5MB/s 


In [4]:
from transformers import BertTokenizer
bert_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_name,
                                          add_special_tokens=True,
                                          do_lower_case=False,
                                          max_length=150,
                                          pad_to_max_length=True)

In [5]:
def bert_encoder(review):
 txt = review.numpy().decode('utf-8')
 encoded = tokenizer.encode_plus(txt, add_special_tokens=True,
                                 max_length=150,
                                 pad_to_max_length=True,
                                 return_attention_mask=True,
                                 truncation=True,
                                 return_token_type_ids=True)
 
 return encoded['input_ids'], encoded['token_type_ids'], encoded['attention_mask']

In [6]:
bert_train = [bert_encoder(r) for r, l in imdb_train]
bert_lbl = [l for r, l in imdb_train]
bert_train = np.array(bert_train)
bert_lbl = tf.keras.utils.to_categorical(bert_lbl, num_classes=2)

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(bert_train,
                                                  bert_lbl,
                                                  test_size=0.2,
                                                  random_state=42)

print(x_train.shape, y_train.shape)

(20000, 3, 150) (20000, 2)


In [8]:
tr_reviews, tr_segments, tr_masks = np.split(x_train, 3, axis=1)
val_reviews, val_segments, val_masks = np.split(x_val, 3, axis=1)

tr_reviews = tr_reviews.squeeze()
tr_segments = tr_segments.squeeze()
tr_masks = tr_masks.squeeze()

val_reviews = val_reviews.squeeze()
val_segments = val_segments.squeeze()
val_masks = val_masks.squeeze()

In [9]:
def example_to_features(input_ids,attention_masks,token_type_ids,y):
 return {"input_ids": input_ids,
         "attention_mask": attention_masks,
         "token_type_ids": token_type_ids},y

In [10]:
train_ds = tf.data.Dataset.from_tensor_slices((tr_reviews,
                                               tr_masks, tr_segments, y_train)).map(example_to_features).shuffle(100).batch(16)

valid_ds = tf.data.Dataset.from_tensor_slices((val_reviews,
                                               val_masks, val_segments, y_val)).map(example_to_features).shuffle(100).batch(16)

In [11]:
from transformers import TFBertForSequenceClassification

bert_model = TFBertForSequenceClassification.from_pretrained(bert_name)

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bert_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
bert_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [14]:
bert_history = bert_model.fit(train_ds, epochs=1, validation_data=valid_ds)

1250/1250 [==============================] - 703s 562ms/step - loss: 0.2771 - accuracy: 0.8846 - val_loss: 0.2569 - val_accuracy: 0.8918


In [ ]:
bert_test = [bert_encoder(r) for r,l in imdb_test]
bert_tst_lbl = [l for r, l in imdb_test]
bert_test2 = np.array(bert_test)

bert_tst_lbl2 = tf.keras.utils.to_categorical (bert_tst_lbl,num_classes=2)

ts_reviews, ts_segments, ts_masks = np.split(bert_test2, 3, axis=1)

ts_reviews = ts_reviews.squeeze()
ts_segments = ts_segments.squeeze()
ts_masks = ts_masks.squeeze()

test_ds = tf.data.Dataset.from_tensor_slices((ts_reviews,
                                              ts_masks, ts_segments, bert_tst_lbl2)).map(example_to_features).shuffle(100).batch(16)


In [15]:
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    bert_model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1



INFO:tensorflow:Assets written to: /tmp/1/assets


INFO:tensorflow:Assets written to: /tmp/1/assets



Saved model:
total 10076
drwxr-xr-x 2 root root     4096 May 21 12:28 assets
-rw-r--r-- 1 root root 10306411 May 21 12:28 saved_model.pb
drwxr-xr-x 2 root root     4096 May 21 12:28 variables


In [16]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [17]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  15994      0 --:--:-- --:--:-- --:--:-- 15994
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Re

In [18]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 72 not upgraded.
Need to get 326 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.5.1 [326 MB]
Fetched 326 MB in 5s (61.8 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.5.1_all.deb ...
Unpacking tensorflow-model-server (2.5.1) ...
Setting up tensorflow-model-server (2.5.1) ...


In [19]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [20]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


Starting job # 0 in a separate thread.


In [21]:
!tail server.log

2021-05-21 12:29:50.182690: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /tmp/1
2021-05-21 12:29:50.420593: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 2125447 microseconds.
2021-05-21 12:29:50.455943: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2021-05-21 12:29:50.456080: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: fashion_model version: 1}
2021-05-21 12:29:50.457244: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-05-21 12:29:50.457308: I tensorflow_serving/model_servers/server.cc:367] Profiler service is enabled
2021-05-21 12:29:50.457755: I tensorflow_serving/model_servers/server.cc:393] Running gRPC ModelServer at 0.0.0.0:8500 

In [52]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": bert_test[0].tolist()})

In [55]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print('Model predictions: {}'.format(predictions))

ConnectionError: ignored

### Custom model with BERT

In [ ]:
from transformers import TFBertModel
bert_name = 'bert-base-cased'
bert = TFBertModel.from_pretrained(bert_name)
bert.summary()

- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
max_seq_len = 150
inp_ids = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="input_ids")
att_mask = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="attention_mask")
seg_ids = tf.keras.layers.Input((max_seq_len,), dtype=tf.int64, name="token_type_ids")

In [ ]:
inp_dict = {"input_ids": inp_ids,
 "attention_mask": att_mask,
 "token_type_ids": seg_ids}
outputs = bert(inp_dict)

'''
  The first output has embeddings for each of the input tokens including the special
  tokens [CLS] and [SEP]. The second output corresponds to the output of the [CLS]
  token. This output will be used further in the mode
'''

bert.trainable = False # don't train BERT any more

In [ ]:
x = tf.keras.layers.Dropout(0.2)(outputs[1])
x = tf.keras.layers.Dense(200, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

custom_model = tf.keras.models.Model(inputs=inp_dict, outputs=x)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
custom_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
custom_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
attention_mask (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_ids (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 150, 768), ( 108310272   attention_mask[0][0]             
                                                                 input_ids[0][0]            

In [ ]:
custom_history = custom_model.fit(train_ds, epochs=3, validation_data=valid_ds)

Epoch 1/3
1250/1250 [==============================] - 304s 234ms/step - loss: 0.4274 - accuracy: 0.8135 - val_loss: 0.3646 - val_accuracy: 0.8358
Epoch 2/3
1250/1250 [==============================] - 290s 232ms/step - loss: 0.3703 - accuracy: 0.8407 - val_loss: 0.3582 - val_accuracy: 0.8376
Epoch 3/3
1250/1250 [==============================] - 291s 232ms/step - loss: 0.3632 - accuracy: 0.8410 - val_loss: 0.3549 - val_accuracy: 0.8378


In [ ]:
custom_model.evaluate(test_ds)